# Overview

In [1]:
from ulkb import *                   # import ULKB Logic namespace
settings.serializer.ulkb.ensure_ascii = True

## Types

In [2]:
from ulkb import *                   # import ULKB Logic namespace
int_tc = TypeConstructor('int', 0)   # create the int t.c.
int_ty = TypeApplication(int_tc)     # create the int type
print(int_ty)                        # ": *" means "is a type"

int : *


In [3]:
list_tc = TypeConstructor('list', 1) # create the list t.c.
list_int_ty = list_tc(int_ty)        # create the (list int) type
print(list_int_ty)

list int : *


## Terms

In [4]:
a = TypeVariable('a')       # create (a : *)
k = Constant('k', a)        # create (k : a)
x = Variable('x', a)        # create (x : a)
id_ = Abstraction(x, x)     # create the identity function
print(id_)

(fun x => x) : a -> a


In [5]:
print(Application(id_, k))  # create the app. of id_ to k

(fun x => x) k : a


In [6]:
id_ = (x >> x)
print(id_)

(fun x => x) : a -> a


In [7]:
print(id_(k))

(fun x => x) k : a


## Deductive system

In [8]:
print(BetaConv(id_(k)))

|- (fun x => x) k = k


In [9]:
x, y = Variables('x', 'y', a)
print((x >> x) == (y >> y))

True


## Logical constants

In [10]:
print(equal(id_))

equal (fun x => x) : (a -> a) -> bool


In [11]:
show_definitions(offset=0, id='true')

4	definition (true : bool) := (fun p => p) = (fun p => p)


In [12]:
def_true = lookup_definition('true')
print(def_true)

|- true <-> (fun p => p) = (fun p => p)


In [13]:
x = Variable('x', BoolType())
print(RuleEqMP(RuleSym(def_true), RuleRefl(x >> x)))

|- true


In [14]:
x, y = Variables('x', 'y', a)
print(forall(Abstraction(x, forall(Abstraction(y, equal(x, y))))))

(forall x y, x = y) : bool


In [15]:
print(Forall(x, y, Equal(x, y)))

(forall x y, x = y) : bool


## Graph API

In [16]:
settings.graph.uri = 'https://query.wikidata.org/sparql'
settings.graph.namespaces = { # Wikidata SPARQL prefixes
   'wd': 'http://www.wikidata.org/entity/',
   'wdt': 'http://www.wikidata.org/prop/direct/'}

In [17]:
a             = TypeVariable('a') # a : *
actor         = new_constant('wd:Q33999', a, label='actor')
blond_hair    = new_constant('wd:Q202466', a, label='blond hair')
female        = new_constant('wd:Q6581072', a, label='female')
Porto_Alegre  = new_constant('wd:Q40269', a, label='Porto Alegre')
singer        = new_constant('wd:Q177220', a, label='singer')

aab           = FunctionType(a, a, BoolType()) # a -> a -> bool : *
date_of_death = new_constant('wdt:P570', aab, label='date of death')
gender        = new_constant('wdt:P21', aab, label='gender')
hair_color    = new_constant('wdt:P1884', aab, label='hair color')
occupation    = new_constant('wdt:P106', aab, label='occupation')
place_of_birth= new_constant('wdt:P19', aab, label='place of birth')

In [18]:
x, y = Variables('x', 'y', a)
q = And(gender(x, female), place_of_birth(x, Porto_Alegre),
        Or(occupation(x, singer), occupation(x, actor)),
        Not(hair_color(x, blond_hair)),
        Exists(y, date_of_death(x, y)))
print(q)

<gender> x <female> and <place of birth> x <Porto Alegre> and (<occupation> x <singer> or <occupation> x <actor>) and not <hair color> x <blond hair> and (exists y, <date of death> x y) : bool


In [19]:
for f in graph.construct(q, q, limit=10):
    new_axiom(f)

In [20]:
show_axioms()

29	axiom <gender> <L\xedlian Lemmertz> <female> and <place of birth> <L\xedlian Lemmertz> <Porto Alegre> and (<occupation> <L\xedlian Lemmertz> <singer> or <occupation> <L\xedlian Lemmertz> <actor>) and not <hair color> <L\xedlian Lemmertz> <blond hair> and (exists y, <date of death> <L\xedlian Lemmertz> y)
30	axiom <gender> <Abigail Maia> <female> and <place of birth> <Abigail Maia> <Porto Alegre> and (<occupation> <Abigail Maia> <singer> or <occupation> <Abigail Maia> <actor>) and not <hair color> <Abigail Maia> <blond hair> and (exists y, <date of death> <Abigail Maia> y)
31	axiom <gender> <Ema D'\xc1vila> <female> and <place of birth> <Ema D'\xc1vila> <Porto Alegre> and (<occupation> <Ema D'\xc1vila> <singer> or <occupation> <Ema D'\xc1vila> <actor>) and not <hair color> <Ema D'\xc1vila> <blond hair> and (exists y, <date of death> <Ema D'\xc1vila> y)
32	axiom <gender> <Elis Regina> <female> and <place of birth> <Elis Regina> <Porto Alegre> and (<occupation> <Elis Regina> <singer> o

In [21]:
Elis_Regina = Constant('wd:Q465877', a, label='Elis Regina')
print(RuleZ3(place_of_birth(Elis_Regina, Porto_Alegre)))

|- <place of birth> <Elis Regina> <Porto Alegre>


In [22]:
Brazil = Constant('wd:Q155', a, label='Brazil')
try:
    print(RuleZ3(place_of_birth(Elis_Regina, Brazil)))
except RuleError as err:
    print(err)

RuleZ3: failed to prove '<place of birth> <Elis Regina> <Brazil> : bool'


In [23]:
for f in graph.paths(Porto_Alegre, x, length=1, limit=200):
    new_axiom(f)

In [24]:
show_axioms(offset=163, limit=3)

163	axiom rdfs:label <Porto Alegre> "Portu Alegri"
164	axiom <flag> <Porto Alegre> <flag of Porto Alegre>
165	axiom <instance of> <Porto Alegre> <port settlement>


In [25]:
country = Constant('wdt:P17', aab, label='country')
x, y, z = Variables('x', 'y', 'z', a)
ax = new_axiom(Forall(x, y, z, Implies(
    place_of_birth(x, y), country(y, z), place_of_birth(x, z))))
print(ax)

|- forall x y z, <place of birth> x y -> <country> y z -> <place of birth> x z


In [26]:
thm = RuleZ3(place_of_birth(Elis_Regina, Brazil))
print(new_theorem(thm))

|- <place of birth> <Elis Regina> <Brazil>


In [27]:
show_theorems()

203	theorem <place of birth> <Elis Regina> <Brazil>
